In [1]:
import keras
import keras.backend as K
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Activation
from keras.optimizers import RMSprop
from keras.callbacks import CSVLogger
import os
import keras.activations
import time

C:\Users\MaksutovRN\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

num_classes = 10

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


In [9]:
batch_size = 64
epochs = 100
units = 64
experiments = 5
start = 0
# activations = ['sigmoid', 'tanh', 'relu', 'linear', 'elu', 'softplus', 'softsign', 'hard_sigmoid', 'LeakyReLU', 'ThresholdedReLU']
activations = ['LeakyReLU']
# activations = ['selu']

In [10]:
start_time = time.time()
counter = 0
total_items = len(activations) * experiments

for act in activations:
    print("Training for activation " + act + " with RMSProp optimizer")

    for i in range(experiments):

        K.clear_session()
        K.reset_uids()

        act_dict = {
          'sigmoid': Activation(keras.activations.sigmoid),
          'tanh': Activation(keras.activations.tanh),
          'relu': Activation(keras.activations.relu),
          'linear': Activation(keras.activations.linear),
          'elu': Activation(keras.activations.elu),
          'softplus': Activation(keras.activations.softplus),
          'softsign': Activation(keras.activations.softsign),
          'hard_sigmoid': Activation(keras.activations.hard_sigmoid),
          'LeakyReLU': keras.layers.advanced_activations.LeakyReLU(alpha=1.5),
          'selu': Activation(keras.activations.selu),
          'ThresholdedReLU': keras.layers.advanced_activations.ThresholdedReLU(theta=0.7) 
        }

        model_name = 'normd_' + act + '_1.5_rmsp_' + str(i + start) + '_' + str(units)
        inputs = Input(shape=(784,))
        x = Dense(units)(inputs)
        x = act_dict[act](x)
        x = Dropout(0.2)(x)
        x = Dense(units)(x)
        x = act_dict[act](x)
        x = Dropout(0.2)(x)
        predictions = Dense(num_classes, activation='softmax')(x)
        model = Model(inputs=inputs, outputs=predictions)

        model.compile(loss='categorical_crossentropy',
                          optimizer=RMSprop(lr=0.001),
                          metrics=['accuracy'])

        csv_logger = CSVLogger('./training_logs/%s.csv' % (model_name), append=False)
        history = model.fit(x_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=1,
                            validation_data=(x_test, y_test), callbacks=[csv_logger])

        score = model.evaluate(x_test, y_test, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        t = time.time()
        time_diff = t - start_time
        counter +=1
        rem_items = total_items - counter
        total_time = round((total_items / counter) * time_diff)
        rem_time = round(total_time - time_diff)
        m, s = divmod(rem_time, 60)
        h, m = divmod(m, 60)
        d, h = divmod(h, 24)
        print('Remaining time: %d days %d hours %02d minutes %02d seconds' % (d, h, m, s))

Training for activation LeakyReLU with RMSProp optimizer
Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 4s 70us/step - loss: 0.4537 - acc: 0.8637 - val_loss: 0.2872 - val_acc: 0.9206
Epoch 2/100
60000/60000 [==============================] - 3s 57us/step - loss: 0.3479 - acc: 0.9016 - val_loss: 0.2755 - val_acc: 0.9212
Epoch 3/100
60000/60000 [==============================] - 3s 56us/step - loss: 0.3253 - acc: 0.9073 - val_loss: 0.2753 - val_acc: 0.9239
Epoch 4/100
60000/60000 [==============================] - 3s 57us/step - loss: 0.3141 - acc: 0.9113 - val_loss: 0.2517 - val_acc: 0.9301
Epoch 5/100
60000/60000 [==============================] - 3s 53us/step - loss: 0.3039 - acc: 0.9142 - val_loss: 0.2548 - val_acc: 0.9287
Epoch 6/100
60000/60000 [==============================] - 3s 56us/step - loss: 0.2988 - acc: 0.9152 - val_loss: 0.2614 - val_acc: 0.9308
Epoch 7/100
60000/60000 [==============================] - 3s 53u

60000/60000 [==============================] - 3s 48us/step - loss: 0.2479 - acc: 0.9307 - val_loss: 0.2222 - val_acc: 0.9394
Epoch 60/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.2485 - acc: 0.9321 - val_loss: 0.2260 - val_acc: 0.9405
Epoch 61/100
60000/60000 [==============================] - 3s 51us/step - loss: 0.2493 - acc: 0.9314 - val_loss: 0.2178 - val_acc: 0.9414
Epoch 62/100
60000/60000 [==============================] - 3s 54us/step - loss: 0.2515 - acc: 0.9309 - val_loss: 0.2204 - val_acc: 0.9397
Epoch 63/100
60000/60000 [==============================] - 3s 55us/step - loss: 0.2505 - acc: 0.9309 - val_loss: 0.2238 - val_acc: 0.9405
Epoch 64/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.2449 - acc: 0.9312 - val_loss: 0.2308 - val_acc: 0.9393
Epoch 65/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.2482 - acc: 0.9311 - val_loss: 0.2279 - val_acc: 0.9392
Epoch 66/100
60000/60000 [==============

Epoch 17/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.2644 - acc: 0.9254 - val_loss: 0.2261 - val_acc: 0.9377
Epoch 18/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.2639 - acc: 0.9257 - val_loss: 0.2213 - val_acc: 0.9379
Epoch 19/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.2624 - acc: 0.9256 - val_loss: 0.2325 - val_acc: 0.9354
Epoch 20/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.2626 - acc: 0.9258 - val_loss: 0.2211 - val_acc: 0.9370
Epoch 21/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.2619 - acc: 0.9256 - val_loss: 0.2299 - val_acc: 0.9387
Epoch 22/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.2590 - acc: 0.9274 - val_loss: 0.2219 - val_acc: 0.9380
Epoch 23/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.2607 - acc: 0.9272 - val_loss: 0.2300 - val_acc: 0.9369
Epoch 24/100
60000/60000 [=

Epoch 76/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.2470 - acc: 0.9300 - val_loss: 0.2430 - val_acc: 0.9365
Epoch 77/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.2476 - acc: 0.9308 - val_loss: 0.2235 - val_acc: 0.9400
Epoch 78/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.2465 - acc: 0.9317 - val_loss: 0.2234 - val_acc: 0.9420
Epoch 79/100
60000/60000 [==============================] - 3s 51us/step - loss: 0.2480 - acc: 0.9317 - val_loss: 0.2323 - val_acc: 0.9388
Epoch 80/100
60000/60000 [==============================] - 3s 51us/step - loss: 0.2488 - acc: 0.9299 - val_loss: 0.2229 - val_acc: 0.9400
Epoch 81/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.2498 - acc: 0.9300 - val_loss: 0.2271 - val_acc: 0.9390
Epoch 82/100
60000/60000 [==============================] - 3s 45us/step - loss: 0.2451 - acc: 0.9315 - val_loss: 0.2266 - val_acc: 0.9404
Epoch 83/100
60000/60000 [=

Epoch 34/100
60000/60000 [==============================] - 3s 56us/step - loss: 0.2554 - acc: 0.9285 - val_loss: 0.2291 - val_acc: 0.9389
Epoch 35/100
60000/60000 [==============================] - 3s 56us/step - loss: 0.2546 - acc: 0.9279 - val_loss: 0.2248 - val_acc: 0.9391
Epoch 36/100
60000/60000 [==============================] - 3s 56us/step - loss: 0.2534 - acc: 0.9288 - val_loss: 0.2211 - val_acc: 0.9387
Epoch 37/100
60000/60000 [==============================] - 3s 56us/step - loss: 0.2543 - acc: 0.9287 - val_loss: 0.2165 - val_acc: 0.9429
Epoch 38/100
60000/60000 [==============================] - 3s 56us/step - loss: 0.2522 - acc: 0.9305 - val_loss: 0.2304 - val_acc: 0.9374
Epoch 39/100
60000/60000 [==============================] - 3s 58us/step - loss: 0.2556 - acc: 0.9283 - val_loss: 0.2253 - val_acc: 0.9390
Epoch 40/100
60000/60000 [==============================] - 4s 62us/step - loss: 0.2538 - acc: 0.9287 - val_loss: 0.2263 - val_acc: 0.9382
Epoch 41/100
60000/60000 [=

Epoch 93/100
60000/60000 [==============================] - 2s 37us/step - loss: 0.2482 - acc: 0.9309 - val_loss: 0.2212 - val_acc: 0.9407
Epoch 94/100
60000/60000 [==============================] - 2s 37us/step - loss: 0.2465 - acc: 0.9309 - val_loss: 0.2174 - val_acc: 0.9427
Epoch 95/100
60000/60000 [==============================] - 2s 37us/step - loss: 0.2456 - acc: 0.9316 - val_loss: 0.2304 - val_acc: 0.9405
Epoch 96/100
60000/60000 [==============================] - 2s 37us/step - loss: 0.2485 - acc: 0.9302 - val_loss: 0.2223 - val_acc: 0.9412
Epoch 97/100
60000/60000 [==============================] - 2s 40us/step - loss: 0.2457 - acc: 0.9318 - val_loss: 0.2206 - val_acc: 0.9406
Epoch 98/100
60000/60000 [==============================] - 2s 38us/step - loss: 0.2431 - acc: 0.9322 - val_loss: 0.2264 - val_acc: 0.9386
Epoch 99/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.2460 - acc: 0.9301 - val_loss: 0.2256 - val_acc: 0.9423
Epoch 100/100
60000/60000 [

Epoch 51/100
60000/60000 [==============================] - 3s 44us/step - loss: 0.2532 - acc: 0.9290 - val_loss: 0.2237 - val_acc: 0.9402
Epoch 52/100
60000/60000 [==============================] - 3s 44us/step - loss: 0.2510 - acc: 0.9289 - val_loss: 0.2265 - val_acc: 0.9401
Epoch 53/100
60000/60000 [==============================] - 2s 41us/step - loss: 0.2481 - acc: 0.9307 - val_loss: 0.2329 - val_acc: 0.9354
Epoch 54/100
60000/60000 [==============================] - 2s 40us/step - loss: 0.2505 - acc: 0.9293 - val_loss: 0.2251 - val_acc: 0.9391
Epoch 55/100
60000/60000 [==============================] - 2s 41us/step - loss: 0.2502 - acc: 0.9298 - val_loss: 0.2267 - val_acc: 0.9407
Epoch 56/100
60000/60000 [==============================] - 2s 42us/step - loss: 0.2513 - acc: 0.9300 - val_loss: 0.2268 - val_acc: 0.9407
Epoch 57/100
60000/60000 [==============================] - 2s 41us/step - loss: 0.2477 - acc: 0.9302 - val_loss: 0.2260 - val_acc: 0.9403
Epoch 58/100
60000/60000 [=

Epoch 9/100
60000/60000 [==============================] - 2s 37us/step - loss: 0.2808 - acc: 0.9203 - val_loss: 0.2279 - val_acc: 0.9355
Epoch 10/100
60000/60000 [==============================] - 2s 38us/step - loss: 0.2789 - acc: 0.9229 - val_loss: 0.2495 - val_acc: 0.9293
Epoch 11/100
60000/60000 [==============================] - 3s 42us/step - loss: 0.2767 - acc: 0.9231 - val_loss: 0.2334 - val_acc: 0.9328
Epoch 12/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.2750 - acc: 0.9226 - val_loss: 0.2325 - val_acc: 0.9329
Epoch 13/100
60000/60000 [==============================] - 2s 40us/step - loss: 0.2701 - acc: 0.9242 - val_loss: 0.2293 - val_acc: 0.9369
Epoch 14/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.2695 - acc: 0.9239 - val_loss: 0.2315 - val_acc: 0.9369
Epoch 15/100
60000/60000 [==============================] - 2s 37us/step - loss: 0.2680 - acc: 0.9243 - val_loss: 0.2330 - val_acc: 0.9356
Epoch 16/100
60000/60000 [==

Epoch 68/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.2466 - acc: 0.9305 - val_loss: 0.2320 - val_acc: 0.9410
Epoch 69/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.2490 - acc: 0.9313 - val_loss: 0.2243 - val_acc: 0.9406
Epoch 70/100
60000/60000 [==============================] - 2s 37us/step - loss: 0.2465 - acc: 0.9305 - val_loss: 0.2418 - val_acc: 0.9363
Epoch 71/100
60000/60000 [==============================] - 2s 40us/step - loss: 0.2453 - acc: 0.9323 - val_loss: 0.2223 - val_acc: 0.9401
Epoch 72/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.2449 - acc: 0.9313 - val_loss: 0.2290 - val_acc: 0.9401
Epoch 73/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.2473 - acc: 0.9312 - val_loss: 0.2268 - val_acc: 0.9406
Epoch 74/100
60000/60000 [==============================] - 2s 40us/step - loss: 0.2492 - acc: 0.9304 - val_loss: 0.2174 - val_acc: 0.9423
Epoch 75/100
60000/60000 [=